# 1. Understand the dataset

## Import dataset

In [1]:
# import libraries and set configs

# inbuilt libs
from collections import Counter
from pprint import pprint

# data analysis
import pandas as pd
import numpy as np

# visualizations
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib.pyplot as plt

# sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, roc_curve, auc 
from sklearn.svm import SVC

# pandas configs
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [2]:
# load data
bids_df = pd.read_csv(r'data/bids.csv')
train_df = pd.read_csv(r'data/train.csv')
test_df = pd.read_csv(r'data/test.csv')
#sample_submission_df = pd.read_csv('data/sample-submission.csv')

## How many rows/columns in each dataset?

In [3]:
print("Train dataset has {} samples and {} attributes".format(*train_df.shape))
print("Test dataset has {} samples and {} attributes".format(*test_df.shape))
print("Bids dataset has {} samples and {} attributes".format(*bids_df.shape))

Train dataset has 2013 samples and 4 attributes
Test dataset has 4700 samples and 3 attributes
Bids dataset has 7656334 samples and 9 attributes


## Exploration of individual datasets

### bids_df exploration

In [4]:
bids_df.head()

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0,c6faf9e526c72bc8383f2bc4417a465d2fa27,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3
1,1,d96e93359ed70b7b4eebac163f448afeb50dc,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c
2,2,065bdcc596308d1fdd0014261d6e198aa138e,wa00e,home goods,phone2,9759243157894736,py,112.54.208.157,vasstdc27m7nks3
3,3,e97aa310037dd210f52b5b164077a926bb5de,jefix,jewelry,phone4,9759243157894736,in,18.99.175.133,vasstdc27m7nks3
4,4,41a7f56fcc5771d305baa6e989fb2e8cceb21,jefix,jewelry,phone5,9759243157894736,in,145.138.5.37,vasstdc27m7nks3


In [5]:
bids_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7656334 entries, 0 to 7656333
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   bid_id       7656334 non-null  int64 
 1   bidder_id    7656334 non-null  object
 2   auction      7656334 non-null  object
 3   merchandise  7656334 non-null  object
 4   device       7656334 non-null  object
 5   time         7656334 non-null  int64 
 6   country      7647475 non-null  object
 7   ip           7656334 non-null  object
 8   url          7656334 non-null  object
dtypes: int64(2), object(7)
memory usage: 525.7+ MB


In [6]:
# proportion of outcome classes
round(bids_df['country'].isnull().mean() * 100, 2)

0.12

In [7]:
# bidder_ids freq count 
bids_df["bidder_id"].value_counts()

e41a3e528811e5c6ff9a6035cd70b2058a416    515033
02ea0f9eb4efeba873bf205908f5e7efb11d3    236413
0afc4c0e477f56b5cef8c63b542ef05d575ce    192565
850e51e72d86ecc2d5899e6512594f5ccb52c    168995
88d76c39cfe85fdbce48714132e2dcef5d515    161935
                                          ...  
2282f5c51bed60c4c7f573f1922e18fdcb4e9         1
6b1ce3da37d1d6124d710b76e8d7c72ab92b4         1
8237e91da835450f6ca53405da9cce9921a5e         1
4393ca629767b7e69b053d1c07930f2b7c728         1
51234b9287b5f31c781c5a6d537f163000e6a         1
Name: bidder_id, Length: 6614, dtype: int64

<b>Observations</b>

* bids_df has 8859 missing country values. This represents 0.12% of the data which may be negligible
* are bots clustered in certain auctions and merchandise (auction site campaign) types?
* there are more devices than bidder_ids, do bots use more devices than humans?
* the num of unique timestamps are much lesser than bid_ids, do bots do multiple bids on exactly the same timestamp?
* are there multiple bidder_ids from the same ip?
* are bots clustered around the same urls?

In [8]:
# num unique values per column
bids_df.nunique()

bid_id         7656334
bidder_id         6614
auction          15051
merchandise         10
device            7351
time            776529
country            199
ip             2303991
url            1786351
dtype: int64

### train_df exploration

In [9]:
train_df.head()

,bidder_id,payment_account,address,outcome
0,4a791121f1d9c9c192d13051301984388c491,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0
1,6feeeab451fcc546e1c43867e04bd5d2294fc,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0
2,0bdfae6168380fa2e00853fd3d6199ba8ed7d,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0
3,3b4586382b3ff164e8bad37b15a268a540996,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0
4,10c7f4bdc67a50bab856b9ca76a83ce980f96,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0


In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2013 entries, 0 to 2012
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   bidder_id        2013 non-null   object 
 1   payment_account  2013 non-null   object 
 2   address          2013 non-null   object 
 3   outcome          2013 non-null   float64
dtypes: float64(1), object(3)
memory usage: 63.0+ KB


In [11]:
train_df.nunique()

bidder_id          2013
payment_account    2013
address            2013
outcome               2
dtype: int64

In [12]:
# num of humans vs bots in train_df
outcome_counter = Counter(train_df['outcome'])

print(f'train_df outcome count: {outcome_counter}')
print(f'proportion of bots {round(outcome_counter[0.0]/outcome_counter[1.0], 2)}%')

train_df outcome count: Counter({0.0: 1910, 1.0: 103})
proportion of bots 18.54%


<b>Observations</b>

* payment_account and address are all unique values (not useful features). 
* Only 18.54% of the train dataset are bots indicating data imbalance. Resampling would be required to balanced out the num of bots vs humans.

### test_df exploration

In [13]:
test_df.head()

,bidder_id,payment_account,address
0,b892f38403f2cc95c1ba1bf4f3b46d89505ea,a3d2de7675556553a5f08e4c88d2c228htx90,5d9fa1b71f992e7c7a106ce4b07a0a754le7c
1,77849f7bfc8bf29c595b50d3640bee1f2189b,a3d2de7675556553a5f08e4c88d2c228rs17i,a3d2de7675556553a5f08e4c88d2c228klidn
2,ef3e34da1e7470d99ed3ce8617e24b6b4541f,925381cce086b8cc9594eee1c77edf665zjpl,a3d2de7675556553a5f08e4c88d2c228aght0
3,7aeb6c9fda0c521f54111c47737e17543aaca,a3d2de7675556553a5f08e4c88d2c228nclv5,b5714de1fd69d4a0d2e39d59e53fe9e15vwat
4,6394924262e3b28e89822b95b170cde4240d8,a3d2de7675556553a5f08e4c88d2c228dtdkd,c3b363a3c3b838d58c85acf0fc9964cb4pnfa


In [14]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4700 entries, 0 to 4699
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   bidder_id        4700 non-null   object
 1   payment_account  4700 non-null   object
 2   address          4700 non-null   object
dtypes: object(3)
memory usage: 110.3+ KB


### some bidders do not have bid history in bids.csv

In [15]:
#Check num of unique bidders in dataset
print(f"bids_df: {bids_df['bidder_id'].nunique()}")
print(f"train_df: {train_df['bidder_id'].nunique()}")
print(f"test_df: {test_df['bidder_id'].nunique()}")

bids_df: 6614
train_df: 2013
test_df: 4700


In [16]:
bids_bidder_ids = bids_df['bidder_id'].unique()
train_bidder_ids = train_df['bidder_id'].unique()
test_bidder_ids = test_df['bidder_id'].unique()

filt_missing_train_bidder_ids = np.isin(train_bidder_ids, bids_bidder_ids, invert=True)
filt_missing_test_bidder_ids = np.isin(test_bidder_ids, bids_bidder_ids, invert=True)

print(f'{filt_missing_train_bidder_ids.sum()} bidder ids from train.csv are not found in bids.csv')
print(f'{filt_missing_test_bidder_ids.sum()} bidder ids from test.csv are not found in bids.csv')

29 bidder ids from train.csv are not found in bids.csv
70 bidder ids from test.csv are not found in bids.csv


# 2. Feature Engineering

Hypothesis:  Bots will participate in more auctions as their purpose is to bid.  

#### Bids count per bidder

In [17]:
count_bids_per_bidder = bids_df.groupby('bidder_id')['auction'].count().reset_index(name='bids_count')
count_bids_per_bidder

,bidder_id,bids_count
0,0013f9b8d1f462df4462e1c1216e441ba6e18,8
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,7
2,002828f800c5132e297cfe3d44fbde9aeac51,2
3,00402f50c4086f320cb6bf94f04462ea32441,249
4,0040cea6b93afd86768c365d89513ffb7c0ba,31
...,...,...
6609,ffd49be672b1ba493b07dccf29311045c5392,168
6610,ffdd8ed91a683b0f2a8237798ebe9214e3c43,5
6611,ffde8717e9a547d536a4e9c3f44782829c021,24
6612,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,3


In [18]:
train_df_new = train_df.merge(count_bids_per_bidder, on='bidder_id', how='left')
train_df_new

,bidder_id,payment_account,address,outcome,bids_count
0,4a791121f1d9c9c192d13051301984388c491,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0
1,6feeeab451fcc546e1c43867e04bd5d2294fc,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0
2,0bdfae6168380fa2e00853fd3d6199ba8ed7d,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0
3,3b4586382b3ff164e8bad37b15a268a540996,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0
4,10c7f4bdc67a50bab856b9ca76a83ce980f96,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0
...,...,...,...,...,...
2008,c51d8ffc9d930dc9a5415dc46120f49c88267,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,36.0
2009,6118a86e8b9eddcfb712642cc966f46de217a,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0
2010,4944331d51a39cbe882cd9f5fc11f4fffa2c0,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,2.0
2011,2ac59d862a484670bf4c4a8abd4161a955b38,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0


In [19]:
test_df_new = test_df.merge(count_bids_per_bidder, on='bidder_id', how='left')
test_df_new.head() #see first 5 records of test_df_new

,bidder_id,payment_account,address,bids_count
0,b892f38403f2cc95c1ba1bf4f3b46d89505ea,a3d2de7675556553a5f08e4c88d2c228htx90,5d9fa1b71f992e7c7a106ce4b07a0a754le7c,4.0
1,77849f7bfc8bf29c595b50d3640bee1f2189b,a3d2de7675556553a5f08e4c88d2c228rs17i,a3d2de7675556553a5f08e4c88d2c228klidn,3.0
2,ef3e34da1e7470d99ed3ce8617e24b6b4541f,925381cce086b8cc9594eee1c77edf665zjpl,a3d2de7675556553a5f08e4c88d2c228aght0,17.0
3,7aeb6c9fda0c521f54111c47737e17543aaca,a3d2de7675556553a5f08e4c88d2c228nclv5,b5714de1fd69d4a0d2e39d59e53fe9e15vwat,148.0
4,6394924262e3b28e89822b95b170cde4240d8,a3d2de7675556553a5f08e4c88d2c228dtdkd,c3b363a3c3b838d58c85acf0fc9964cb4pnfa,23.0


---

Hypothesis: Unique count may be of interest as bots may participate in more unique auctions, use more unique devices, ips and urls and be in more unique countries.   

#### Unique auctions count

In [20]:
count_uniq_auction_per_bidder = bids_df.groupby('bidder_id')['auction'].nunique().reset_index(name='unique_auction_count')
count_uniq_auction_per_bidder

,bidder_id,unique_auction_count
0,0013f9b8d1f462df4462e1c1216e441ba6e18,6
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,7
2,002828f800c5132e297cfe3d44fbde9aeac51,2
3,00402f50c4086f320cb6bf94f04462ea32441,89
4,0040cea6b93afd86768c365d89513ffb7c0ba,23
...,...,...
6609,ffd49be672b1ba493b07dccf29311045c5392,20
6610,ffdd8ed91a683b0f2a8237798ebe9214e3c43,4
6611,ffde8717e9a547d536a4e9c3f44782829c021,16
6612,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,3


In [21]:
train_df_new = train_df_new.merge(count_uniq_auction_per_bidder, on='bidder_id', how='left')
train_df_new.head()

,bidder_id,payment_account,address,outcome,bids_count,unique_auction_count
0,4a791121f1d9c9c192d13051301984388c491,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0
1,6feeeab451fcc546e1c43867e04bd5d2294fc,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0
2,0bdfae6168380fa2e00853fd3d6199ba8ed7d,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0
3,3b4586382b3ff164e8bad37b15a268a540996,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0
4,10c7f4bdc67a50bab856b9ca76a83ce980f96,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0


In [22]:
test_df_new = test_df_new.merge(count_uniq_auction_per_bidder, on='bidder_id', how='left')
test_df_new.head()

,bidder_id,payment_account,address,bids_count,unique_auction_count
0,b892f38403f2cc95c1ba1bf4f3b46d89505ea,a3d2de7675556553a5f08e4c88d2c228htx90,5d9fa1b71f992e7c7a106ce4b07a0a754le7c,4.0,3.0
1,77849f7bfc8bf29c595b50d3640bee1f2189b,a3d2de7675556553a5f08e4c88d2c228rs17i,a3d2de7675556553a5f08e4c88d2c228klidn,3.0,2.0
2,ef3e34da1e7470d99ed3ce8617e24b6b4541f,925381cce086b8cc9594eee1c77edf665zjpl,a3d2de7675556553a5f08e4c88d2c228aght0,17.0,14.0
3,7aeb6c9fda0c521f54111c47737e17543aaca,a3d2de7675556553a5f08e4c88d2c228nclv5,b5714de1fd69d4a0d2e39d59e53fe9e15vwat,148.0,90.0
4,6394924262e3b28e89822b95b170cde4240d8,a3d2de7675556553a5f08e4c88d2c228dtdkd,c3b363a3c3b838d58c85acf0fc9964cb4pnfa,23.0,20.0


#### Unique merchandise count

In [23]:
count_uniq_merchandise_per_bidder = bids_df.groupby('bidder_id')['merchandise'].nunique().reset_index(name='unique_merchandise_count')
count_uniq_merchandise_per_bidder

,bidder_id,unique_merchandise_count
0,0013f9b8d1f462df4462e1c1216e441ba6e18,1
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,1
2,002828f800c5132e297cfe3d44fbde9aeac51,1
3,00402f50c4086f320cb6bf94f04462ea32441,1
4,0040cea6b93afd86768c365d89513ffb7c0ba,1
...,...,...
6609,ffd49be672b1ba493b07dccf29311045c5392,1
6610,ffdd8ed91a683b0f2a8237798ebe9214e3c43,1
6611,ffde8717e9a547d536a4e9c3f44782829c021,1
6612,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,1


In [24]:
train_df_new = train_df_new.merge(count_uniq_merchandise_per_bidder, on='bidder_id', how='left')
train_df_new

,bidder_id,payment_account,address,outcome,bids_count,unique_auction_count,unique_merchandise_count
0,4a791121f1d9c9c192d13051301984388c491,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,1.0
1,6feeeab451fcc546e1c43867e04bd5d2294fc,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,1.0
2,0bdfae6168380fa2e00853fd3d6199ba8ed7d,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0
3,3b4586382b3ff164e8bad37b15a268a540996,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0
4,10c7f4bdc67a50bab856b9ca76a83ce980f96,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,1.0
...,...,...,...,...,...,...,...
2008,c51d8ffc9d930dc9a5415dc46120f49c88267,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,36.0,25.0,1.0
2009,6118a86e8b9eddcfb712642cc966f46de217a,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0
2010,4944331d51a39cbe882cd9f5fc11f4fffa2c0,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,2.0,1.0,1.0
2011,2ac59d862a484670bf4c4a8abd4161a955b38,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0


In [25]:
test_df_new = test_df_new.merge(count_uniq_merchandise_per_bidder, on='bidder_id', how='left')
test_df_new.head()

,bidder_id,payment_account,address,bids_count,unique_auction_count,unique_merchandise_count
0,b892f38403f2cc95c1ba1bf4f3b46d89505ea,a3d2de7675556553a5f08e4c88d2c228htx90,5d9fa1b71f992e7c7a106ce4b07a0a754le7c,4.0,3.0,1.0
1,77849f7bfc8bf29c595b50d3640bee1f2189b,a3d2de7675556553a5f08e4c88d2c228rs17i,a3d2de7675556553a5f08e4c88d2c228klidn,3.0,2.0,1.0
2,ef3e34da1e7470d99ed3ce8617e24b6b4541f,925381cce086b8cc9594eee1c77edf665zjpl,a3d2de7675556553a5f08e4c88d2c228aght0,17.0,14.0,1.0
3,7aeb6c9fda0c521f54111c47737e17543aaca,a3d2de7675556553a5f08e4c88d2c228nclv5,b5714de1fd69d4a0d2e39d59e53fe9e15vwat,148.0,90.0,1.0
4,6394924262e3b28e89822b95b170cde4240d8,a3d2de7675556553a5f08e4c88d2c228dtdkd,c3b363a3c3b838d58c85acf0fc9964cb4pnfa,23.0,20.0,1.0


#### Unique devices count

In [26]:
count_uniq_devices_per_bidder = bids_df.groupby('bidder_id')['device'].nunique().reset_index(name='unique_devices_count')
count_uniq_devices_per_bidder

,bidder_id,unique_devices_count
0,0013f9b8d1f462df4462e1c1216e441ba6e18,7
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,7
2,002828f800c5132e297cfe3d44fbde9aeac51,1
3,00402f50c4086f320cb6bf94f04462ea32441,99
4,0040cea6b93afd86768c365d89513ffb7c0ba,21
...,...,...
6609,ffd49be672b1ba493b07dccf29311045c5392,69
6610,ffdd8ed91a683b0f2a8237798ebe9214e3c43,3
6611,ffde8717e9a547d536a4e9c3f44782829c021,20
6612,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,2


In [27]:
train_df_new = train_df_new.merge(count_uniq_devices_per_bidder, on='bidder_id', how='left')
train_df_new

,bidder_id,payment_account,address,outcome,bids_count,unique_auction_count,unique_merchandise_count,unique_devices_count
0,4a791121f1d9c9c192d13051301984388c491,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,1.0,14.0
1,6feeeab451fcc546e1c43867e04bd5d2294fc,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,1.0,2.0
2,0bdfae6168380fa2e00853fd3d6199ba8ed7d,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,2.0
3,3b4586382b3ff164e8bad37b15a268a540996,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0
4,10c7f4bdc67a50bab856b9ca76a83ce980f96,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,1.0,53.0
...,...,...,...,...,...,...,...,...
2008,c51d8ffc9d930dc9a5415dc46120f49c88267,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,36.0,25.0,1.0,4.0
2009,6118a86e8b9eddcfb712642cc966f46de217a,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0
2010,4944331d51a39cbe882cd9f5fc11f4fffa2c0,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,2.0,1.0,1.0,2.0
2011,2ac59d862a484670bf4c4a8abd4161a955b38,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0


In [28]:
test_df_new = test_df_new.merge(count_uniq_devices_per_bidder, on='bidder_id', how='left')

#### Unique time count

In [29]:
count_uniq_time_per_bidder = bids_df.groupby('bidder_id')['time'].nunique().reset_index(name='unique_time_count')
count_uniq_time_per_bidder

,bidder_id,unique_time_count
0,0013f9b8d1f462df4462e1c1216e441ba6e18,8
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,7
2,002828f800c5132e297cfe3d44fbde9aeac51,2
3,00402f50c4086f320cb6bf94f04462ea32441,249
4,0040cea6b93afd86768c365d89513ffb7c0ba,31
...,...,...
6609,ffd49be672b1ba493b07dccf29311045c5392,168
6610,ffdd8ed91a683b0f2a8237798ebe9214e3c43,5
6611,ffde8717e9a547d536a4e9c3f44782829c021,24
6612,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,3


In [30]:
train_df_new = train_df_new.merge(count_uniq_time_per_bidder, on='bidder_id', how='left')
train_df_new

,bidder_id,payment_account,address,outcome,bids_count,unique_auction_count,unique_merchandise_count,unique_devices_count,unique_time_count
0,4a791121f1d9c9c192d13051301984388c491,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,1.0,14.0,24.0
1,6feeeab451fcc546e1c43867e04bd5d2294fc,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,1.0,2.0,3.0
2,0bdfae6168380fa2e00853fd3d6199ba8ed7d,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,2.0,4.0
3,3b4586382b3ff164e8bad37b15a268a540996,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0
4,10c7f4bdc67a50bab856b9ca76a83ce980f96,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,1.0,53.0,155.0
...,...,...,...,...,...,...,...,...,...
2008,c51d8ffc9d930dc9a5415dc46120f49c88267,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,36.0,25.0,1.0,4.0,33.0
2009,6118a86e8b9eddcfb712642cc966f46de217a,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0
2010,4944331d51a39cbe882cd9f5fc11f4fffa2c0,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,2.0,1.0,1.0,2.0,2.0
2011,2ac59d862a484670bf4c4a8abd4161a955b38,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0


In [31]:
test_df_new = test_df_new.merge(count_uniq_time_per_bidder, on='bidder_id', how='left')

#### Unique country count

In [32]:
count_uniq_country_per_bidder = bids_df.groupby('bidder_id')['country'].nunique().reset_index(name='unique_country_count')
count_uniq_country_per_bidder

,bidder_id,unique_country_count
0,0013f9b8d1f462df4462e1c1216e441ba6e18,2
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,4
2,002828f800c5132e297cfe3d44fbde9aeac51,1
3,00402f50c4086f320cb6bf94f04462ea32441,21
4,0040cea6b93afd86768c365d89513ffb7c0ba,6
...,...,...
6609,ffd49be672b1ba493b07dccf29311045c5392,5
6610,ffdd8ed91a683b0f2a8237798ebe9214e3c43,1
6611,ffde8717e9a547d536a4e9c3f44782829c021,6
6612,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,2


In [33]:
train_df_new = train_df_new.merge(count_uniq_country_per_bidder, on='bidder_id', how='left')
train_df_new

,bidder_id,payment_account,address,outcome,bids_count,unique_auction_count,unique_merchandise_count,unique_devices_count,unique_time_count,unique_country_count
0,4a791121f1d9c9c192d13051301984388c491,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,1.0,14.0,24.0,6.0
1,6feeeab451fcc546e1c43867e04bd5d2294fc,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,1.0,2.0,3.0,1.0
2,0bdfae6168380fa2e00853fd3d6199ba8ed7d,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,2.0,4.0,1.0
3,3b4586382b3ff164e8bad37b15a268a540996,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,10c7f4bdc67a50bab856b9ca76a83ce980f96,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,1.0,53.0,155.0,2.0
...,...,...,...,...,...,...,...,...,...,...
2008,c51d8ffc9d930dc9a5415dc46120f49c88267,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,36.0,25.0,1.0,4.0,33.0,4.0
2009,6118a86e8b9eddcfb712642cc966f46de217a,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2010,4944331d51a39cbe882cd9f5fc11f4fffa2c0,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,2.0,1.0,1.0,2.0,2.0,1.0
2011,2ac59d862a484670bf4c4a8abd4161a955b38,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [34]:
test_df_new = test_df_new.merge(count_uniq_country_per_bidder, on='bidder_id', how='left')

#### Unique IP count 

In [35]:
count_uniq_ip_per_bidder = bids_df.groupby('bidder_id')['ip'].nunique().reset_index(name='unique_ip_count')
count_uniq_ip_per_bidder

,bidder_id,unique_ip_count
0,0013f9b8d1f462df4462e1c1216e441ba6e18,8
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,7
2,002828f800c5132e297cfe3d44fbde9aeac51,2
3,00402f50c4086f320cb6bf94f04462ea32441,151
4,0040cea6b93afd86768c365d89513ffb7c0ba,28
...,...,...
6609,ffd49be672b1ba493b07dccf29311045c5392,102
6610,ffdd8ed91a683b0f2a8237798ebe9214e3c43,4
6611,ffde8717e9a547d536a4e9c3f44782829c021,24
6612,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,3


In [36]:
train_df_new = train_df_new.merge(count_uniq_ip_per_bidder, on='bidder_id', how='left')
train_df_new

,bidder_id,payment_account,address,outcome,bids_count,unique_auction_count,unique_merchandise_count,unique_devices_count,unique_time_count,unique_country_count,unique_ip_count
0,4a791121f1d9c9c192d13051301984388c491,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,1.0,14.0,24.0,6.0,20.0
1,6feeeab451fcc546e1c43867e04bd5d2294fc,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,1.0,2.0,3.0,1.0,3.0
2,0bdfae6168380fa2e00853fd3d6199ba8ed7d,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,2.0,4.0,1.0,4.0
3,3b4586382b3ff164e8bad37b15a268a540996,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,10c7f4bdc67a50bab856b9ca76a83ce980f96,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,1.0,53.0,155.0,2.0,123.0
...,...,...,...,...,...,...,...,...,...,...,...
2008,c51d8ffc9d930dc9a5415dc46120f49c88267,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,36.0,25.0,1.0,4.0,33.0,4.0,5.0
2009,6118a86e8b9eddcfb712642cc966f46de217a,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2010,4944331d51a39cbe882cd9f5fc11f4fffa2c0,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0
2011,2ac59d862a484670bf4c4a8abd4161a955b38,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [37]:
test_df_new = test_df_new.merge(count_uniq_ip_per_bidder, on='bidder_id', how='left')

#### Unique URL count

In [38]:
count_uniq_url_per_bidder = bids_df.groupby('bidder_id')['url'].nunique().reset_index(name='unique_url_count')
count_uniq_url_per_bidder

,bidder_id,unique_url_count
0,0013f9b8d1f462df4462e1c1216e441ba6e18,4
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,4
2,002828f800c5132e297cfe3d44fbde9aeac51,1
3,00402f50c4086f320cb6bf94f04462ea32441,98
4,0040cea6b93afd86768c365d89513ffb7c0ba,4
...,...,...
6609,ffd49be672b1ba493b07dccf29311045c5392,40
6610,ffdd8ed91a683b0f2a8237798ebe9214e3c43,2
6611,ffde8717e9a547d536a4e9c3f44782829c021,4
6612,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,2


In [39]:
train_df_new = train_df_new.merge(count_uniq_url_per_bidder, on='bidder_id', how='left')
train_df_new

,bidder_id,payment_account,address,outcome,bids_count,unique_auction_count,unique_merchandise_count,unique_devices_count,unique_time_count,unique_country_count,unique_ip_count,unique_url_count
0,4a791121f1d9c9c192d13051301984388c491,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,1.0,14.0,24.0,6.0,20.0,1.0
1,6feeeab451fcc546e1c43867e04bd5d2294fc,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,1.0,2.0,3.0,1.0,3.0,2.0
2,0bdfae6168380fa2e00853fd3d6199ba8ed7d,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,2.0,4.0,1.0,4.0,2.0
3,3b4586382b3ff164e8bad37b15a268a540996,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,10c7f4bdc67a50bab856b9ca76a83ce980f96,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,1.0,53.0,155.0,2.0,123.0,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2008,c51d8ffc9d930dc9a5415dc46120f49c88267,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,36.0,25.0,1.0,4.0,33.0,4.0,5.0,2.0
2009,6118a86e8b9eddcfb712642cc966f46de217a,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2010,4944331d51a39cbe882cd9f5fc11f4fffa2c0,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0
2011,2ac59d862a484670bf4c4a8abd4161a955b38,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [40]:
test_df_new = test_df_new.merge(count_uniq_url_per_bidder, on='bidder_id', how='left')

---

If unique count may be of interest, unique count per auction may be also be of interest.

#### Average number of unique devices per auction

In [41]:
mean_uniq_devices_per_auction = bids_df.groupby(['bidder_id','auction'])['device'].nunique().groupby('bidder_id').mean().reset_index(name='mean_unique_devices_per_auction')
mean_uniq_devices_per_auction

,bidder_id,mean_unique_devices_per_auction
0,0013f9b8d1f462df4462e1c1216e441ba6e18,1.333333
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,1.000000
2,002828f800c5132e297cfe3d44fbde9aeac51,1.000000
3,00402f50c4086f320cb6bf94f04462ea32441,1.943820
4,0040cea6b93afd86768c365d89513ffb7c0ba,1.304348
...,...,...
6609,ffd49be672b1ba493b07dccf29311045c5392,6.150000
6610,ffdd8ed91a683b0f2a8237798ebe9214e3c43,1.250000
6611,ffde8717e9a547d536a4e9c3f44782829c021,1.500000
6612,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,1.000000


In [42]:
train_df_new = train_df_new.merge(mean_uniq_devices_per_auction, on='bidder_id', how='left')
train_df_new

,bidder_id,payment_account,address,outcome,bids_count,unique_auction_count,unique_merchandise_count,unique_devices_count,unique_time_count,unique_country_count,unique_ip_count,unique_url_count,mean_unique_devices_per_auction
0,4a791121f1d9c9c192d13051301984388c491,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,1.0,14.0,24.0,6.0,20.0,1.0,1.333333
1,6feeeab451fcc546e1c43867e04bd5d2294fc,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,1.0,2.0,3.0,1.0,3.0,2.0,2.000000
2,0bdfae6168380fa2e00853fd3d6199ba8ed7d,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,2.0,4.0,1.0,4.0,2.0,1.000000
3,3b4586382b3ff164e8bad37b15a268a540996,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000
4,10c7f4bdc67a50bab856b9ca76a83ce980f96,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,1.0,53.0,155.0,2.0,123.0,91.0,3.434783
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,c51d8ffc9d930dc9a5415dc46120f49c88267,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,36.0,25.0,1.0,4.0,33.0,4.0,5.0,2.0,1.000000
2009,6118a86e8b9eddcfb712642cc966f46de217a,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000
2010,4944331d51a39cbe882cd9f5fc11f4fffa2c0,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,2.000000
2011,2ac59d862a484670bf4c4a8abd4161a955b38,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000


In [43]:
test_df_new = test_df_new.merge(mean_uniq_devices_per_auction, on='bidder_id', how='left')

#### Average number of unique IPs per auction

In [44]:
mean_uniq_ips_per_auction = bids_df.groupby(['bidder_id','auction'])['ip'].nunique().groupby('bidder_id').mean().reset_index(name='mean_unique_ips_per_auction')
mean_uniq_ips_per_auction

,bidder_id,mean_unique_ips_per_auction
0,0013f9b8d1f462df4462e1c1216e441ba6e18,1.333333
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,1.000000
2,002828f800c5132e297cfe3d44fbde9aeac51,1.000000
3,00402f50c4086f320cb6bf94f04462ea32441,2.280899
4,0040cea6b93afd86768c365d89513ffb7c0ba,1.304348
...,...,...
6609,ffd49be672b1ba493b07dccf29311045c5392,7.100000
6610,ffdd8ed91a683b0f2a8237798ebe9214e3c43,1.250000
6611,ffde8717e9a547d536a4e9c3f44782829c021,1.500000
6612,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,1.000000


In [45]:
train_df_new = train_df_new.merge(mean_uniq_ips_per_auction, on='bidder_id', how='left')
train_df_new

,bidder_id,payment_account,address,outcome,bids_count,unique_auction_count,unique_merchandise_count,unique_devices_count,unique_time_count,unique_country_count,unique_ip_count,unique_url_count,mean_unique_devices_per_auction,mean_unique_ips_per_auction
0,4a791121f1d9c9c192d13051301984388c491,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,1.0,14.0,24.0,6.0,20.0,1.0,1.333333,1.333333
1,6feeeab451fcc546e1c43867e04bd5d2294fc,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,1.0,2.0,3.0,1.0,3.0,2.0,2.000000,3.000000
2,0bdfae6168380fa2e00853fd3d6199ba8ed7d,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,2.0,4.0,1.0,4.0,2.0,1.000000,1.000000
3,3b4586382b3ff164e8bad37b15a268a540996,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000
4,10c7f4bdc67a50bab856b9ca76a83ce980f96,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,1.0,53.0,155.0,2.0,123.0,91.0,3.434783,5.608696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,c51d8ffc9d930dc9a5415dc46120f49c88267,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,36.0,25.0,1.0,4.0,33.0,4.0,5.0,2.0,1.000000,1.000000
2009,6118a86e8b9eddcfb712642cc966f46de217a,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000
2010,4944331d51a39cbe882cd9f5fc11f4fffa2c0,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,2.000000,2.000000
2011,2ac59d862a484670bf4c4a8abd4161a955b38,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000


In [46]:
test_df_new = test_df_new.merge(mean_uniq_ips_per_auction, on='bidder_id', how='left')

#### Average number of unique URLs per auction

In [47]:
mean_uniq_urls_per_auction = bids_df.groupby(['bidder_id','auction'])['url'].nunique().groupby('bidder_id').mean().reset_index(name='mean_unique_urls_per_auction')
mean_uniq_urls_per_auction

,bidder_id,mean_unique_urls_per_auction
0,0013f9b8d1f462df4462e1c1216e441ba6e18,1.166667
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,1.000000
2,002828f800c5132e297cfe3d44fbde9aeac51,1.000000
3,00402f50c4086f320cb6bf94f04462ea32441,2.089888
4,0040cea6b93afd86768c365d89513ffb7c0ba,1.304348
...,...,...
6609,ffd49be672b1ba493b07dccf29311045c5392,4.350000
6610,ffdd8ed91a683b0f2a8237798ebe9214e3c43,1.000000
6611,ffde8717e9a547d536a4e9c3f44782829c021,1.125000
6612,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,1.000000


In [48]:
train_df_new = train_df_new.merge(mean_uniq_urls_per_auction, on='bidder_id', how='left')
train_df_new

,bidder_id,payment_account,address,outcome,bids_count,unique_auction_count,unique_merchandise_count,unique_devices_count,unique_time_count,unique_country_count,unique_ip_count,unique_url_count,mean_unique_devices_per_auction,mean_unique_ips_per_auction,mean_unique_urls_per_auction
0,4a791121f1d9c9c192d13051301984388c491,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,1.0,14.0,24.0,6.0,20.0,1.0,1.333333,1.333333,1.000000
1,6feeeab451fcc546e1c43867e04bd5d2294fc,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,1.0,2.0,3.0,1.0,3.0,2.0,2.000000,3.000000,2.000000
2,0bdfae6168380fa2e00853fd3d6199ba8ed7d,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,2.0,4.0,1.0,4.0,2.0,1.000000,1.000000,1.000000
3,3b4586382b3ff164e8bad37b15a268a540996,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000
4,10c7f4bdc67a50bab856b9ca76a83ce980f96,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,1.0,53.0,155.0,2.0,123.0,91.0,3.434783,5.608696,4.695652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,c51d8ffc9d930dc9a5415dc46120f49c88267,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,36.0,25.0,1.0,4.0,33.0,4.0,5.0,2.0,1.000000,1.000000,1.000000
2009,6118a86e8b9eddcfb712642cc966f46de217a,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000
2010,4944331d51a39cbe882cd9f5fc11f4fffa2c0,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,2.000000,2.000000,1.000000
2011,2ac59d862a484670bf4c4a8abd4161a955b38,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000


In [49]:
test_df_new = test_df_new.merge(mean_uniq_urls_per_auction, on='bidder_id', how='left')

---

How many bids (aggregated) were made with the same ip / same url per auction?

#### Average number of bids per IP per auction

In [50]:
mean_bids_per_ip_auction = bids_df.groupby(['bidder_id','auction', 'ip'])['bid_id'].count().groupby('bidder_id').mean().reset_index(name='mean_bids_per_ip_per_auction')
mean_bids_per_ip_auction

,bidder_id,mean_bids_per_ip_per_auction
0,0013f9b8d1f462df4462e1c1216e441ba6e18,1.000000
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,1.000000
2,002828f800c5132e297cfe3d44fbde9aeac51,1.000000
3,00402f50c4086f320cb6bf94f04462ea32441,1.226601
4,0040cea6b93afd86768c365d89513ffb7c0ba,1.033333
...,...,...
6609,ffd49be672b1ba493b07dccf29311045c5392,1.183099
6610,ffdd8ed91a683b0f2a8237798ebe9214e3c43,1.000000
6611,ffde8717e9a547d536a4e9c3f44782829c021,1.000000
6612,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,1.000000


In [51]:
train_df_new = train_df_new.merge(mean_bids_per_ip_auction, on='bidder_id', how='left')
train_df_new

,bidder_id,payment_account,address,outcome,bids_count,unique_auction_count,unique_merchandise_count,unique_devices_count,unique_time_count,unique_country_count,unique_ip_count,unique_url_count,mean_unique_devices_per_auction,mean_unique_ips_per_auction,mean_unique_urls_per_auction,mean_bids_per_ip_per_auction
0,4a791121f1d9c9c192d13051301984388c491,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,1.0,14.0,24.0,6.0,20.0,1.0,1.333333,1.333333,1.000000,1.00000
1,6feeeab451fcc546e1c43867e04bd5d2294fc,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,1.0,2.0,3.0,1.0,3.0,2.0,2.000000,3.000000,2.000000,1.00000
2,0bdfae6168380fa2e00853fd3d6199ba8ed7d,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,2.0,4.0,1.0,4.0,2.0,1.000000,1.000000,1.000000,1.00000
3,3b4586382b3ff164e8bad37b15a268a540996,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.00000
4,10c7f4bdc67a50bab856b9ca76a83ce980f96,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,1.0,53.0,155.0,2.0,123.0,91.0,3.434783,5.608696,4.695652,1.20155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,c51d8ffc9d930dc9a5415dc46120f49c88267,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,36.0,25.0,1.0,4.0,33.0,4.0,5.0,2.0,1.000000,1.000000,1.000000,1.44000
2009,6118a86e8b9eddcfb712642cc966f46de217a,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.00000
2010,4944331d51a39cbe882cd9f5fc11f4fffa2c0,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,2.000000,2.000000,1.000000,1.00000
2011,2ac59d862a484670bf4c4a8abd4161a955b38,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.00000


In [52]:
test_df_new = test_df_new.merge(mean_bids_per_ip_auction, on='bidder_id', how='left')

#### Average number of bids per URL per auction

In [53]:
mean_bids_per_url_auction = bids_df.groupby(['bidder_id','auction', 'url'])['bid_id'].count().groupby('bidder_id').mean().reset_index(name='mean_bids_per_url_per_auction')
mean_bids_per_url_auction

,bidder_id,mean_bids_per_url_per_auction
0,0013f9b8d1f462df4462e1c1216e441ba6e18,1.142857
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,1.000000
2,002828f800c5132e297cfe3d44fbde9aeac51,1.000000
3,00402f50c4086f320cb6bf94f04462ea32441,1.338710
4,0040cea6b93afd86768c365d89513ffb7c0ba,1.033333
...,...,...
6609,ffd49be672b1ba493b07dccf29311045c5392,1.931034
6610,ffdd8ed91a683b0f2a8237798ebe9214e3c43,1.250000
6611,ffde8717e9a547d536a4e9c3f44782829c021,1.333333
6612,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,1.000000


In [54]:
train_df_new = train_df_new.merge(mean_bids_per_url_auction, on='bidder_id', how='left')
train_df_new

,bidder_id,payment_account,address,outcome,bids_count,unique_auction_count,unique_merchandise_count,unique_devices_count,unique_time_count,unique_country_count,unique_ip_count,unique_url_count,mean_unique_devices_per_auction,mean_unique_ips_per_auction,mean_unique_urls_per_auction,mean_bids_per_ip_per_auction,mean_bids_per_url_per_auction
0,4a791121f1d9c9c192d13051301984388c491,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,1.0,14.0,24.0,6.0,20.0,1.0,1.333333,1.333333,1.000000,1.00000,1.333333
1,6feeeab451fcc546e1c43867e04bd5d2294fc,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,1.0,2.0,3.0,1.0,3.0,2.0,2.000000,3.000000,2.000000,1.00000,1.500000
2,0bdfae6168380fa2e00853fd3d6199ba8ed7d,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,2.0,4.0,1.0,4.0,2.0,1.000000,1.000000,1.000000,1.00000,1.000000
3,3b4586382b3ff164e8bad37b15a268a540996,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.00000,1.000000
4,10c7f4bdc67a50bab856b9ca76a83ce980f96,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,1.0,53.0,155.0,2.0,123.0,91.0,3.434783,5.608696,4.695652,1.20155,1.435185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,c51d8ffc9d930dc9a5415dc46120f49c88267,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,36.0,25.0,1.0,4.0,33.0,4.0,5.0,2.0,1.000000,1.000000,1.000000,1.44000,1.440000
2009,6118a86e8b9eddcfb712642cc966f46de217a,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.00000,1.000000
2010,4944331d51a39cbe882cd9f5fc11f4fffa2c0,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,2.000000,2.000000,1.000000,1.00000,2.000000
2011,2ac59d862a484670bf4c4a8abd4161a955b38,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.00000,1.000000


In [55]:
test_df_new = test_df_new.merge(mean_bids_per_url_auction, on='bidder_id', how='left')

<hr>

Hypothesis: Bots may place lesser bids in the auction to win the bid.

#### Average number of bids per auction

In [56]:
mean_bids_per_auction_per_bidder = bids_df.groupby(['bidder_id', 'auction'])['bid_id'].count().groupby('bidder_id').mean().reset_index(name='mean_bids_per_auction')
mean_bids_per_auction_per_bidder

,bidder_id,mean_bids_per_auction
0,0013f9b8d1f462df4462e1c1216e441ba6e18,1.333333
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,1.000000
2,002828f800c5132e297cfe3d44fbde9aeac51,1.000000
3,00402f50c4086f320cb6bf94f04462ea32441,2.797753
4,0040cea6b93afd86768c365d89513ffb7c0ba,1.347826
...,...,...
6609,ffd49be672b1ba493b07dccf29311045c5392,8.400000
6610,ffdd8ed91a683b0f2a8237798ebe9214e3c43,1.250000
6611,ffde8717e9a547d536a4e9c3f44782829c021,1.500000
6612,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,1.000000


In [57]:
train_df_new = train_df_new.merge(mean_bids_per_auction_per_bidder, on='bidder_id', how='left')
train_df_new

,bidder_id,payment_account,address,outcome,bids_count,unique_auction_count,unique_merchandise_count,unique_devices_count,unique_time_count,unique_country_count,unique_ip_count,unique_url_count,mean_unique_devices_per_auction,mean_unique_ips_per_auction,mean_unique_urls_per_auction,mean_bids_per_ip_per_auction,mean_bids_per_url_per_auction,mean_bids_per_auction
0,4a791121f1d9c9c192d13051301984388c491,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,1.0,14.0,24.0,6.0,20.0,1.0,1.333333,1.333333,1.000000,1.00000,1.333333,1.333333
1,6feeeab451fcc546e1c43867e04bd5d2294fc,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,1.0,2.0,3.0,1.0,3.0,2.0,2.000000,3.000000,2.000000,1.00000,1.500000,3.000000
2,0bdfae6168380fa2e00853fd3d6199ba8ed7d,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,2.0,4.0,1.0,4.0,2.0,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000
3,3b4586382b3ff164e8bad37b15a268a540996,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000
4,10c7f4bdc67a50bab856b9ca76a83ce980f96,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,1.0,53.0,155.0,2.0,123.0,91.0,3.434783,5.608696,4.695652,1.20155,1.435185,6.739130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,c51d8ffc9d930dc9a5415dc46120f49c88267,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,36.0,25.0,1.0,4.0,33.0,4.0,5.0,2.0,1.000000,1.000000,1.000000,1.44000,1.440000,1.440000
2009,6118a86e8b9eddcfb712642cc966f46de217a,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000
2010,4944331d51a39cbe882cd9f5fc11f4fffa2c0,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,2.000000,2.000000,1.000000,1.00000,2.000000,2.000000
2011,2ac59d862a484670bf4c4a8abd4161a955b38,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [58]:
test_df_new = test_df_new.merge(mean_bids_per_auction_per_bidder, on='bidder_id', how='left')

---

### Clean the data

In [59]:
train_df_cleaned = train_df_new.fillna(0)
test_df_cleaned = test_df_new.fillna(0)

In [60]:
print(train_df_cleaned.info())
print(test_df_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2013 entries, 0 to 2012
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   bidder_id                        2013 non-null   object 
 1   payment_account                  2013 non-null   object 
 2   address                          2013 non-null   object 
 3   outcome                          2013 non-null   float64
 4   bids_count                       2013 non-null   float64
 5   unique_auction_count             2013 non-null   float64
 6   unique_merchandise_count         2013 non-null   float64
 7   unique_devices_count             2013 non-null   float64
 8   unique_time_count                2013 non-null   float64
 9   unique_country_count             2013 non-null   float64
 10  unique_ip_count                  2013 non-null   float64
 11  unique_url_count                 2013 non-null   float64
 12  mean_unique_devices_

# 3. Train the model

### 3a. Select the features

In [63]:
 features = ['bids_count',
             'unique_auction_count',
             'unique_devices_count', 'unique_time_count', 'unique_country_count',
             'unique_ip_count', 'unique_url_count',
             'mean_unique_devices_per_auction', 'mean_unique_ips_per_auction',
             'mean_unique_urls_per_auction', 'mean_bids_per_ip_per_auction',
             'mean_bids_per_url_per_auction', 'mean_bids_per_auction'] 

    
X = train_df_cleaned[features]
y = train_df_cleaned['outcome']

X_kaggle = test_df_cleaned[features] 

### 3b. Import the models

#### Random Forest Classifier

In [ ]:
# rf_model = RandomForestClassifier(random_state=5) 
# features = ['bids_count',
#              'unique_auction_count', 'unique_merchandise_count',
#              'unique_devices_count', 'unique_time_count', 'unique_country_count',
#              'unique_ip_count', 'unique_url_count',
#              'mean_unique_devices_per_auction', 'mean_unique_ips_per_auction',
#              'mean_unique_urls_per_auction', 'mean_bids_per_ip_per_auction',
#              'mean_bids_per_url_per_auction', 'mean_bids_per_auction']
# AUC=0.900

In [64]:
# train-test split used (consider k-fold )

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Instantiate the model object
rf_model = RandomForestClassifier(random_state=42)

# Fit the model with the training data
rf_model = rf_model.fit(X_train, y_train)

# Predict the target on the test dataset
y_pred_probabilities = rf_model.predict_proba(X_test)[:, 1]

# Calculate the (1) false positive rate, (2) true positive rate, and (3) thresholds
y_pred_probabilities = list(y_pred_probabilities) # converting to list
fpr, tpr, thresholds = roc_curve(y_test, y_pred_probabilities)

# Plotting the chart
fig = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curve (AUC={auc(fpr, tpr):.3f})',
    labels=dict(x='1 - Specificity', y='Sensitivity'),
    width=700, height=500
)

# This part is just for formatting & adding the dash-line 
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')


fig.show()

Evaulate random forest model to determine parameters

In [65]:
# Look at parameters used by random forest classifier
print('Parameters currently in use:\n')
pprint(rf_model.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [66]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


### After looking at the available parameters in the random forest classifer model, we proceed to select six parameters to adjust.

In [67]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf_model = RandomForestClassifier(random_state=42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf_model, param_distributions=random_grid,
                              n_iter = 100, scoring='neg_mean_absolute_error', 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(X_train, y_train);

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [68]:
# to retrieve best parameters from random search
rf_random.best_params_

{'n_estimators': 600,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 110,
 'bootstrap': False}

## Grid Search building is done after random search was completed for parameters searching.

In [69]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

# Create a base model
rf = RandomForestClassifier (random_state = 42)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)

In [70]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train);

Fitting 3 folds for each of 288 candidates, totalling 864 fits


In [71]:
# to obtain best parameters from GridSearch
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 80,
 'max_features': 2,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 300}

# 4. Generate the predictions on the test dataset for submission

### 4a. Retrain the model on all the train dataset after finding optimised parameters

In [72]:
# Instantiate the model object with optimised parameters
# optimal used in v1 -> ('bootstrap'=True, 'max_depth': 80, 'max_features': 3, 'min_samples_leaf': 3,'min_samples_split': 8,'n_estimators': 100)
# randomised used in v2 ->  'bootstrap': False,  'max_depth': 10, 'max_features': 'sqrt','min_samples_leaf': 2, 'min_samples_split': 5,, 'n_estimators': 1600,

rf_submission_model = RandomForestClassifier(bootstrap=False, max_depth=10, max_features='sqrt', min_samples_leaf=2,min_samples_split=5,n_estimators=1600)

# Fit the model with the training data
rf_submission_model = rf_submission_model.fit(X, y)

### 4b. Use model for prediction

In [73]:
results = rf_submission_model.predict_proba(X_kaggle)
results

array([[9.99995681e-01, 4.31910569e-06],
       [9.47878710e-01, 5.21212905e-02],
       [9.99839573e-01, 1.60426720e-04],
       ...,
       [9.52495381e-01, 4.75046186e-02],
       [9.99992940e-01, 7.06033376e-06],
       [9.85794217e-01, 1.42057831e-02]])

In [74]:
kaggle_preds = results[:,1]  # Extract values from the rightmost column
kaggle_preds

array([4.31910569e-06, 5.21212905e-02, 1.60426720e-04, ...,
       4.75046186e-02, 7.06033376e-06, 1.42057831e-02])

### 4c. Write to csv for submission

In [ ]:
submission_df = pd.DataFrame({
    'Id': test_df_cleaned['bidder_id'],
    'Predicted': kaggle_preds
})
submission_df.to_csv("submission_28112021_02.csv", index=False)  